# Answering Business Question using SQL

In [30]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Overview of the Data

In [31]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [32]:
%%sql
SELECT *
FROM artist
LIMIT 5;

Done.


artist_id,name
1,AC/DC
2,Accept
3,Aerosmith
4,Alanis Morissette
5,Alice In Chains


In [33]:
%%sql
SELECT *
FROM invoice_line
LIMIT 5;

Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
2,1,1159,0.99,1
3,1,1160,0.99,1
4,1,1161,0.99,1
5,1,1162,0.99,1


In [34]:
%%sql
SELECT *
FROM genre
LIMIT 5;

Done.


genre_id,name
1,Rock
2,Jazz
3,Metal
4,Alternative & Punk
5,Rock And Roll


## Selecting Albums to Purchase

We need to figure out which genres sell most tracks in the USA and make a recommendation of which albums we should purchase for the store.

In [35]:
#genre(genre_id) - track(genre_id) - invoice_line(track_id) 

In [36]:
%%sql


SELECT
    g.name genre,
    COUNT(il.track_id) number_of_tracks,
    COUNT(CAST(il.track_id AS float))/ CAST((
                                            SELECT COUNT(track_id)
                                            FROM invoice_line)
                                             AS float) percentage_from_total
    
        
FROM genre g
INNER JOIN track t ON t.genre_id = g.genre_id
INNER JOIN invoice_line il ON il.track_id = t.track_id
GROUP BY 1
ORDER BY 2 DESC;

Done.


genre,number_of_tracks,percentage_from_total
Rock,2635,0.553920538154299
Metal,619,0.13012402774858103
Alternative & Punk,492,0.10342652932520496
Latin,167,0.03510615934412445
R&B/Soul,159,0.033424427159974776
Blues,124,0.026066848854319948
Jazz,121,0.025436199285263823
Alternative,117,0.024595333193188986
Easy Listening,74,0.015556022703384485
Pop,63,0.013243640950178683


Based on the analysis of the tracks sold by genre in our database, the most popular genre is Rock, taking 55% from total amount of tracks sold.
From the list of four albums that are to be added to the store (Hip-Hop, Punk, Pop, Blues) Blues, Pop nd Hip-hop/Rap are more popular and it is recommended to purchase them for the store. The genre Punk, however, did not appear in the table, therefore, we can assume that there were no albums in that genre.

## Analyzing Employee Sales Performance

We are going to find out the total dollar amount of sales assigned to each sales support agent within the company.

In [37]:
%%sql
    
SELECT 
    c.support_rep_id,
    SUM(i.total) sales,
    e.*
FROM customer c
INNER JOIN invoice i ON i.customer_id = c.customer_id
INNER JOIN employee e ON e.employee_id = c.support_rep_id
GROUP BY 1
ORDER BY 1;


Done.


support_rep_id,sales,employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
3,1731.510000000004,3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,1584.0000000000032,4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,1393.9200000000028,5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


Writing a quiery above shows that there are three sales support agents. Sales Support Agent 3 has biggest sales compared to other two. However, looking at hire date for each we can observe that the sales agent 3 was hired first. The sales representative with lowest amount of sales was hired last, 6 month after the first sales representative. Threfore, we can assume that the difference in sales is expalined by the less time spent working within the company.


## Analyzing Sales by Country

In [38]:
%%sql
WITH other_countries AS
    (SELECT 
        c.country,
        COUNT(distinct c.customer_id) count,
        SUM(il.unit_price) total_sales,
        SUM(il.unit_price)/COUNT(distinct c.customer_id) avg_sales_per_customer,
        SUM(il.unit_price)/COUNT(distinct i.invoice_id) avg_order,
        
        CASE 
            WHEN COUNT(distinct c.customer_id) = 1 THEN "Others"
            ELSE c.country
        END AS countries
    FROM customer c
    INNER JOIN invoice i ON c.customer_id = i.customer_id
    INNER JOIN invoice_line il ON i.invoice_id = il.invoice_id
     
    GROUP BY 1
    )                
    
SELECT  
    countries,
    SUM(count),
    total_sales,
    avg_sales_per_customer,
    avg_order
     
                         
FROM
    (SELECT 
         o.*,
         CASE
             WHEN countries = "Others" THEN 1
             ELSE 0
         END AS sort
     FROM other_countries o
    )

GROUP BY 1
ORDER BY sort ASC, total_sales DESC;



Done.


countries,SUM(count),total_sales,avg_sales_per_customer,avg_order
USA,13,1040.490000000008,80.03769230769292,7.942671755725252
Canada,8,535.5900000000034,66.94875000000043,7.047236842105309
Brazil,5,427.68000000000245,85.53600000000048,7.011147540983647
France,5,389.0700000000021,77.81400000000042,7.781400000000042
Germany,4,334.6200000000016,83.6550000000004,8.161463414634186
Czech Republic,2,273.24000000000103,136.62000000000052,9.108000000000034
United Kingdom,3,245.52000000000078,81.84000000000026,8.768571428571457
Portugal,2,185.13000000000022,92.56500000000011,6.3837931034482835
India,2,183.1500000000002,91.5750000000001,8.72142857142858
Others,15,75.23999999999998,75.23999999999998,7.523999999999998


Based on the results we can see that most of the customers come from the USA, taking majority of total sales. However, by the terms of the average sale per customer, Czech Republic is leading in the table. Moreover, average order figures are alos highest for Check Republic. Another perspective country is India, with second highest average order. 

## Albums vs Individual Tracks

In [47]:
%%sql

WITH tracks_sold AS
    (SELECT 
        il.invoice_id,
        t.album_id,
        COUNT(il.track_id) tracks_sold
    FROM invoice_line il
    INNER JOIN track t ON t.track_id = il.track_id
    GROUP BY 1, 2),
    
    tracks_in_album AS
     (SELECT 
       album_id,
       COUNT(track_id)
    FROM track
    GROUP BY 1)
        




(sqlite3.OperationalError) near ";": syntax error
[SQL: WITH tracks_sold AS
    (SELECT 
        il.invoice_id,
        t.album_id,
        COUNT(il.track_id) tracks_sold
    FROM invoice_line il
    INNER JOIN track t ON t.track_id = il.track_id
    GROUP BY 1, 2),
    
    tracks_in_album AS
     (SELECT 
       album_id,
       COUNT(track_id)
    FROM track
    GROUP BY 1);]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [48]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;


Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Album purchases account for 18.6% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.